In [ ]:
import ROOT
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib.gridspec as gridspec
import math
import uproot
from array import array

In [ ]:
#data_file = "/Users/alexanderantonakis/Software/SBNDWireModAna/Data/wiremod_ndhist_widths_data_xf.root"
#mc_file = "/Users/alexanderantonakis/Software/SBNDWireModAna/Data/wiremod_ndwidths_mc_xf.root"

# Larger Stats
data_file = "/Users/alexanderantonakis/Software/SBNDWireModAna/Data/wiremod_ndwidths_data_xf_6k.root"
mc_file = "/Users/alexanderantonakis/Software/SBNDWireModAna/Data/wiremod_ndwidths_mc_xf_6k.root"

print("Got MC and Data Files to compare and analyze")

In [ ]:
dim_dict = {0: "x", 1: "y", 2: "z", 3:"txz", 4:"tyz", 5:"dqdx"}
label_dict = {0: "Reconstructed Hit X [cm]", 
              1: "Reconstructed Hit Y [cm]", 
              2: "Reconstructed Hit Z [cm]", 
              3:"Reconstructed Hit "+r"$\theta_{XZ}$ [degrees]", 
              4:"Reconstructed Hit "+r"$\theta_{YZ}$ [degrees]", 
              5:"Reconstructed Hit dQ/dx [ADC/cm]"}

In [ ]:
rfile_data = ROOT.TFile.Open(data_file)
rfile_mc = ROOT.TFile.Open(mc_file)

DATA_TAG = "v10_06_00_02"

rfile_data.ls()
rfile_data.Close()

In [ ]:
# Function to profile a 2D histogram along X axis and compute iterative truncated mean for each slice

def get_XY(file_name, tpc, plane):
    idx = 3*tpc+plane
    rfile = ROOT.TFile.Open(file_name)
    h = rfile.Get("hwidth"+str(idx))

    # Loop over the bins in the selected dimension
    #for bin in range(1, h.GetAxis(dim).GetNbins() + 1):
        #h.GetAxis(dim).SetRange(bin, bin)

    h2d = h.Projection(1, 0)
    h2d.SetDirectory(0)
    h.Delete()
    return h2d


print("function defined")                  

In [ ]:
# I want to convert a ROOT TH2D to a plt.hist2d for better plotting
def root_th2d_to_plt_hist2d(th2d):
    # Get the bin edges and contents from the ROOT TH2D
    x_edges = [th2d.GetXaxis().GetBinLowEdge(i) for i in range(1, th2d.GetNbinsX() + 2)]
    y_edges = [th2d.GetYaxis().GetBinLowEdge(i) for i in range(1, th2d.GetNbinsY() + 2)]
    z_values = np.zeros((th2d.GetNbinsX(), th2d.GetNbinsY()))

    for i in range(1, th2d.GetNbinsX() + 1):
        for j in range(1, th2d.GetNbinsY() + 1):
            z_values[i-1, j-1] = th2d.GetBinContent(i, j)

    return np.array(x_edges), np.array(y_edges), np.array(z_values)

# Example usage:
# th2d = rfile_data.Get("hwidth0")
# x_edges, y_edges, z_values = root_th2d_to_plt_hist2d(th2d)
# plt.hist2d(x_edges[:-1], y_edges[:-1], z_values.T, bins=[x_edges, y_edges], cmap='viridis')
# plt.colorbar(label='Counts')
# plt.xlabel('X-axis label')                
#plt.ylabel('Y-axis label')
# plt.title('2D Histogram from ROOT TH2D')
# plt.show()    

# I want to convert a ROOT TH1D with errors to a plt.errorbar for better plotting
def root_th1d_to_plt_errorbar(th1d):
    # Get the bin centers, contents, and errors from the ROOT TH1D
    x_values = [th1d.GetBinCenter(i) for i in range(1, th1d.GetNbinsX() + 1)]
    y_values = [th1d.GetBinContent(i) for i in range(1, th1d.GetNbinsX() + 1)]
    y_errors = [th1d.GetBinError(i) for i in range(1, th1d.GetNbinsX() + 1)]

    return np.array(x_values), np.array(y_values), np.array(y_errors) 

# Example usage:
# th1d = rfile_data.Get("hwidth0").ProjectionX()
# x_values, y_values, y_errors = root_th1d_to_plt_errorbar(th1d)
# plt.errorbar(x_values, y_values, yerr=y_errors, fmt='o', ecolor='red', capsize=5)
# plt.xlabel('X-axis label')                
# plt.ylabel('Y-axis label')
# plt.title('1D Histogram with Error Bars from ROOT TH1D')
# plt.show()


# Function to create a ROOT TH1D from means and errors
def make_mean_hist(h2d, means, errors):
    h_means = h2d.ProjectionX()
    h_means.Reset()
    for i in range(1, h_means.GetNbinsX() + 1):
        h_means.SetBinContent(i, means[i-1])
        h_means.SetBinError(i, errors[i-1])
    
    return h_means

In [ ]:

for num in range(3):
    th2d = get_XY(data_file, 0, num)

    x_edges, y_edges, z_values = root_th2d_to_plt_hist2d(th2d)
    z_masked = np.ma.masked_where(z_values == 0, z_values)

    cmap = plt.cm.viridis.copy()
    cmap.set_bad(color='white')

         
    X, Y = np.meshgrid(x_edges, y_edges, indexing='ij')  # Needed for pcolormesh
    #ax.pcolormesh(X, Y, z_values, shading='auto', cmap='viridis')
    MAX_Z = np.max(z_values)
         
    #pcm = plt.pcolormesh(X, Y, z_masked, cmap=cmap, shading='auto', norm=LogNorm(vmin=1, vmax=MAX_Z))
    pcm = plt.pcolormesh(X, Y, z_masked, cmap=cmap, shading='auto')
    cbar = plt.colorbar(pcm)

    plt.xlim([-200, 0])
    plt.ylim([-200, 200])
    plt.xlabel("Reconstructed Hit X [cm]", fontsize=14)
    plt.ylabel("Reconstructed Hit Y [cm]", fontsize=14)
    plt.title("Data: East TPC, Plane "+str(num))
    plt.show()

In [ ]:

for num in range(3):
    th2d = get_XY(data_file, 1, num)

    x_edges, y_edges, z_values = root_th2d_to_plt_hist2d(th2d)
    z_masked = np.ma.masked_where(z_values == 0, z_values)

    cmap = plt.cm.viridis.copy()
    cmap.set_bad(color='white')

         
    X, Y = np.meshgrid(x_edges, y_edges, indexing='ij')  # Needed for pcolormesh
    #ax.pcolormesh(X, Y, z_values, shading='auto', cmap='viridis')
    MAX_Z = np.max(z_values)
         
    #pcm = plt.pcolormesh(X, Y, z_masked, cmap=cmap, shading='auto', norm=LogNorm(vmin=1, vmax=MAX_Z))
    pcm = plt.pcolormesh(X, Y, z_masked, cmap=cmap, shading='auto')
    cbar = plt.colorbar(pcm)

    plt.xlim([0, 200])
    plt.ylim([-200, 200])
    plt.xlabel("Reconstructed Hit X [cm]", fontsize=14)
    plt.ylabel("Reconstructed Hit Y [cm]", fontsize=14)
    plt.title("Data: West TPC, Plane "+str(num))
    plt.show()

In [ ]:
# Function to profile a 2D histogram along X axis and compute iterative truncated mean for each slice

def get_ZY(file_name, tpc, plane):
    idx = 3*tpc+plane
    rfile = ROOT.TFile.Open(file_name)
    h = rfile.Get("hwidth"+str(idx))

    # Loop over the bins in the selected dimension
    #for bin in range(1, h.GetAxis(dim).GetNbins() + 1):
        #h.GetAxis(dim).SetRange(bin, bin)

    h2d = h.Projection(1, 2)
    h2d.SetDirectory(0)
    h.Delete()
    return h2d


print("function defined")                  

In [ ]:

for num in range(3):
    th2d = get_ZY(data_file, 0, num)

    x_edges, y_edges, z_values = root_th2d_to_plt_hist2d(th2d)
    z_masked = np.ma.masked_where(z_values == 0, z_values)

    cmap = plt.cm.viridis.copy()
    cmap.set_bad(color='white')

         
    X, Y = np.meshgrid(x_edges, y_edges, indexing='ij')  # Needed for pcolormesh
    #ax.pcolormesh(X, Y, z_values, shading='auto', cmap='viridis')
    MAX_Z = np.max(z_values)
         
    #pcm = plt.pcolormesh(X, Y, z_masked, cmap=cmap, shading='auto', norm=LogNorm(vmin=1, vmax=MAX_Z))
    pcm = plt.pcolormesh(X, Y, z_masked, cmap=cmap, shading='auto')
    cbar = plt.colorbar(pcm)

    plt.xlim([0, 500])
    plt.ylim([-200, 200])
    plt.xlabel("Reconstructed Hit Z [cm]", fontsize=14)
    plt.ylabel("Reconstructed Hit Y [cm]", fontsize=14)
    plt.title("Data: East TPC, Plane "+str(num))
    plt.show()

In [ ]:

for num in range(3):
    th2d = get_ZY(data_file, 1, num)

    x_edges, y_edges, z_values = root_th2d_to_plt_hist2d(th2d)
    z_masked = np.ma.masked_where(z_values == 0, z_values)

    cmap = plt.cm.viridis.copy()
    cmap.set_bad(color='white')

         
    X, Y = np.meshgrid(x_edges, y_edges, indexing='ij')  # Needed for pcolormesh
    #ax.pcolormesh(X, Y, z_values, shading='auto', cmap='viridis')
    MAX_Z = np.max(z_values)
         
    #pcm = plt.pcolormesh(X, Y, z_masked, cmap=cmap, shading='auto', norm=LogNorm(vmin=1, vmax=MAX_Z))
    pcm = plt.pcolormesh(X, Y, z_masked, cmap=cmap, shading='auto')
    cbar = plt.colorbar(pcm)

    plt.xlim([0, 500])
    plt.ylim([-200, 200])
    plt.xlabel("Reconstructed Hit Z [cm]", fontsize=14)
    plt.ylabel("Reconstructed Hit Y [cm]", fontsize=14)
    plt.title("Data: West TPC, Plane "+str(num))
    plt.show()